# Exploratory Data Analysis with SQL

### Our Data
This dataset includes a record for each payload carried during a SpaceX mission

Importing our libraries:

In [1]:
import csv, sqlite3
import pandas as pd

Connecting to the database

In [2]:
# loading SQL extension
%load_ext sql
# establishing connection
con = sqlite3.connect('my_data.db')
cur = con.cursor()

In [3]:
%sql sqlite:///my_data.db

Loading data into database

In [4]:
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

101

Creating new table with blank rows removed.

In [5]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data.db
(sqlite3.OperationalError) table SPACEXTABLE already exists
[SQL: create table SPACEXTABLE as select * from SPACEXTBL where Date is not null]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


Verifying our table was created correctly.

In [6]:
%sql select * from SPACEXTABLE limit 1

 * sqlite:///my_data.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-04-06,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)


## Exploring our data using SQL Magic

#### 1. Displaying names of unique launch sites 

In [7]:
%sql select distinct "Launch_Site" from SPACEXTABLE

 * sqlite:///my_data.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


#### 2. Displaying records where launch sites begin with 'CCA'

In [8]:
%sql select * from SPACEXTABLE where "Launch_Site" like 'CCA%' limit 5

 * sqlite:///my_data.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-04-06,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-08-12,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,07:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-08-10,00:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-01-03,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


#### 3. Displaying total payload mass carried by boosters launched by NASA (CRS)

In [9]:
%sql select sum(PAYLOAD_MASS__KG_) as TOTALPAYLOAD from SPACEXTABLE where "Customer" like '%NASA%CRS%'

 * sqlite:///my_data.db
Done.


TOTALPAYLOAD
48213


#### 4. Displaying average payload mass carried by booster F9 v1.1

In [10]:
%sql select avg(PAYLOAD_MASS__KG_) as AVG_PAYLOAD_MASS from SPACEXTABLE where "Booster_Version" like '%F9 v1.1%'

 * sqlite:///my_data.db
Done.


AVG_PAYLOAD_MASS
2534.6666666666665


#### 5. Displaying date of the first succesful landing outcome in a ground pad

In [11]:
%%sql
select min("Date") as FIRST_SUCCESSFUL_LANDING
from SPACEXTABLE where "Landing_Outcome"="Success (ground pad)"

 * sqlite:///my_data.db
Done.


FIRST_SUCCESSFUL_LANDING
2015-12-22


#### 6. What boosters have success in drone ship and have a payload mass greater than 4000 but less than 6000?

In [12]:
%%sql
select "Booster_Version", PAYLOAD_MASS__KG_ from SPACEXTABLE 
where "Landing_Outcome"="Success (drone ship)" and PAYLOAD_MASS__KG_ between 4000 and 6000

 * sqlite:///my_data.db
Done.


Booster_Version,PAYLOAD_MASS__KG_
F9 FT B1022,4696
F9 FT B1026,4600
F9 FT B1021.2,5300
F9 FT B1031.2,5200


#### 7. Total number of succesful and failure mission outcomes?

In [13]:
%%sql
select 
count(case when "Mission_Outcome"="Success" then "Mission_Outcome" END) as SUCCESS,
count(case when "Mission_Outcome"="Failure (in flight)" then "Mission_Outcome" END) as FAILURE
from SPACEXTABLE 

 * sqlite:///my_data.db
Done.


SUCCESS,FAILURE
98,1


#### 8. What boosters have carried the maximum payload mass?

In [14]:
%%sql 
select "Booster_Version" from SPACEXTABLE 
where PAYLOAD_MASS__KG_ = (select max(PAYLOAD_MASS__KG_) from SPACEXTABLE)

 * sqlite:///my_data.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


#### 9. Records for failure landing_outcomes in drone ship during 2015

In [15]:
%%sql
select
substr(Date,6,2) as MONTH,
substr(Date,0,5) as YEAR,
"Booster_Version",
"Launch_Site",
"Landing_Outcome"
from SPACEXTABLE
where 
substr(Date,0,5)='2015'
and 
"Landing_Outcome"="Failure (drone ship)"

 * sqlite:///my_data.db
Done.


MONTH,YEAR,Booster_Version,Launch_Site,Landing_Outcome
10,2015,F9 v1.1 B1012,CCAFS LC-40,Failure (drone ship)
04,2015,F9 v1.1 B1015,CCAFS LC-40,Failure (drone ship)


#### 10. Ranking the count of landing outcomes between 2010-06-02 and         2017-03-20 (desc)

In [19]:
%%sql
select
"Landing_Outcome",
count(*) as COUNT
from SPACEXTABLE
where "Date" between "2010-06-04" and "2017-03-20"
group by "Landing_Outcome"
order by COUNT desc

 * sqlite:///my_data.db
Done.


Landing_Outcome,COUNT
No attempt,10
Success (ground pad),5
Success (drone ship),5
Failure (drone ship),5
Controlled (ocean),3
Uncontrolled (ocean),2
Precluded (drone ship),1
Failure (parachute),1
